# Data Analysis and Visualization in Python
## Accessing SQLite Databases Using Python and Pandas
Questions
* What if my data are stored in an SQL database? Can I manage them with Python?
* How can I write data from Python to be used with SQL?

Objectives
* Use the sqlite3 module to interact with a SQL database.
* Access data stored in SQLite using Python.
* Describe the difference in interacting with data stored as a CSV file versus in SQLite.
* Describe the benefits of accessing data using a database compared to a CSV file.

## Python and SQL
### The `sqlite3` module

In [ ]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("../../data/portal_mammals.sqlite")

cur = con.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute('SELECT * FROM species;'):
    print(row)

# Be sure to close the connection.
con.close()

### Queries

In [ ]:
# Create a SQL connection to our SQLite database
con = sqlite3.connect("../../data/portal_mammals.sqlite")

cur = con.cursor()

In [ ]:
# Get all table names
cur.execute('SELECT name FROM sqlite_master WHERE type = "table"')
cur.fetchall()

In [ ]:
# Get information about the plots table
cur.execute('PRAGMA table_info(plots)')
cur.fetchall()

In [ ]:
# Get all different plot types and their count, sorted by count
cur.execute('SELECT plot_type,COUNT(plot_type) FROM plots ' +
            'GROUP BY plot_type ' +
            'ORDER BY COUNT(plot_type)')
cur.fetchall()

In [ ]:
# Get all plot IDs where the plot type is "Spectab exclosure"
cur.execute('SELECT plot_id FROM plots WHERE plot_type = "Spectab exclosure"')
cur.fetchall()

In [ ]:
# Return only the first result of a query
cur.execute('SELECT genus,COUNT(genus) FROM surveys ' +
            'INNER JOIN plots   ON plots.plot_id      = surveys.plot ' +
            'INNER JOIN species ON species.species_id = surveys.species ' +
            'GROUP BY genus HAVING plots.plot_type = "Spectab exclosure" ' +
            'ORDER BY COUNT(genus) DESC')
cur.fetchone()

In [ ]:
# Be sure to close the connection.
con.close()

### Exercise - SQL Queries
Create a query that contains survey data collected between 1998 and 2001 for observations of sex "female" or "male", and keep all different combinations of genus, species name and plot type for the sample. How many records are returned?

In [ ]:
con = sqlite3.connect("../../data/portal_mammals.sqlite")
cur = con.cursor()

cur.execute('SELECT DISTINCT genus,species.species,plot_type FROM surveys ' +
            'INNER JOIN plots   ON plots.plot_id      = surveys.plot ' +
            'INNER JOIN species ON species.species_id = surveys.species ' +
            'WHERE (year BETWEEN 1998 AND 2001) ' +
            'AND (sex IN ("F", "M"))')
print(len(cur.fetchall()))

con.close()

## Accessing data stored in SQLite using Python and Pandas

In [ ]:
import pandas as pd

# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../../data/portal_mammals.sqlite")
df = pd.read_sql_query("SELECT * FROM surveys", con)

# Verify that result of SQL query is stored in the dataframe
print(df.head())

con.close()

## Storing data: CSV vs SQLite
Storing your data in an SQLite database can provide substantial performance improvements when reading/writing compared to CSV. The difference in performance becomes more noticeable as the size of the dataset grows (see for example [these benchmarks](http://sebastianraschka.com/Articles/2013_sqlite_database.html#results-and-conclusions)).

### Exercise - SQL and Pandas
Create a dataframe that contains the total number of observations (`COUNT`) and the `SUM` of observed weights (`wgt`) for each site ID (`plot`) for all years, ordered by site ID.

In [ ]:
con = sqlite3.connect("../../data/portal_mammals.sqlite")

df = pd.read_sql_query('SELECT plot,year,COUNT(record_id),SUM(wgt) FROM surveys ' +
                       'GROUP BY plot,year ORDER BY plot', con)

print(df.head())
print(df.tail())

con.close()

## Storing data: Create new tables using Pandas

In [ ]:
con = sqlite3.connect("../../data/portal_mammals.sqlite")

# Load the data into a DataFrame
surveys_df = pd.read_sql_query("SELECT * FROM surveys", con)

# Select only data for 2002
surveys2002 = surveys_df[surveys_df['year'] == 2002]

# Write the new DataFrame to a new SQLite table
surveys2002.to_sql("surveys2002", con, if_exists="replace")

con.close()

### Exercise - Saving your work
1. For each of the challenges in the previous exercise block, modify your code to save the results to their own tables in the portal database.
1. What are some of the reasons you might want to save the results of your queries back into the database? What are some of the reasons you might avoid doing this.